In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import os
import sys
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer
from plot_learning_curves import plot_learning_curves
from weighted_accuracy_and_tools import decompose_y, reconstruct_y


In [3]:
X = os.path.join("..","data","original_input", "X_train_filled.csv")
Y = os.path.join("..","data","original_input", "X_test_GgyECq8.csv")
W = os.path.join("..","data","original_input", "y_train_jJtXgMX.csv")

X_train = pd.read_csv(X, delimiter=',')
X_test = pd.read_csv(Y,delimiter=',')
y_train = pd.read_csv(W, delimiter=',')
delivery_start = X_test["DELIVERY_START"]

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\data\\original_input\\X_train_filled.csv'

In [ ]:
X_train = process_data(X_train.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_test.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(y_train.copy(deep=True), None, None, None)

In [ ]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Initialize the RandomForestRegressor
reg = RandomForestRegressor(n_jobs=-1, **{'min_samples_leaf': 2, 'min_samples_split': 5})

# Set up the parameter grid to search
reg.fit(X_train, y_train_magnitude)
y_pred_magn = reg.predict(X_test)

In [ ]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred_magn.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../../data/output_y_test/y_result_magn_rdf1.csv", index = False)